## Installing Transformers library

In [42]:
#Installing transformer package 
!pip install transformers==4.29.1
!pip install huggingface-hub==0.14.1
!pip install tokenizers==0.13.3 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 38.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.15.1
    Uninstalling huggingface-hub-0.15.1:
      Successfully uninstalled huggingface-hub-0.15.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Rating Estimation by ChatGPT

In [ ]:
!pip install openai

In [ ]:
import os 
import openai
os.environ["OpenAI_API_Key"] = ""
api_key = os.environ.get("OpenAI_API_Key")
openai.api_key = api_key

In [ ]:
# Loss to be added in the custom loss
def get_chatgpt_rating(prompt, sample):
  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", 
     "content": f"{prompt} {sample}"}
  ])
  return 10 - int(completion.choices[0].message)

In [ ]:
# Different prompts tested in order to generate a sensible rating 
prompt1 = "Provide me rating between 0 and 10 (without any explanation), where 0 is the best and 10 is the worst, for the following story summary: " 
prompt2 = "Assign a rating between 0 (best) and 10 (worst) to the given artificial story summary (only give rating as the response):"
prompt3 = "Assign a rating between 0 (best) and 10 (worst) to the given artificial story summary (only give rating as the response). The rating should be based on writing style, coherence and capture strength. Summary:"
prompt4 = "Assign a rating between 0 and 10 to the given artificial story summary. Only give rating as the response (no reasoning). The rating should be based on writing style, coherence, and capture strength. Summary:" # Best
prompt5 = "Provide me rating between 0 and 10 (without any explanation),  for the following story summary:" 

## Data Tokenization/Encoding

In [1]:
#Loading the standard T5 small model and tokenizer 
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Can try different T5 models such as T5-large, T5-3B, T5-11B
base_tokenizer = AutoTokenizer.from_pretrained('t5-base')
base_model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [166]:
# Encoding the sequences
def encode_sequences(x, base_tokenizer = base_tokenizer):
  # try:
    # Input consists of different aspects of the story on which the output will be conditioned
    input = str(x['Input']) 
    # Label is the conditioned output - Story
    label = str(x['Summary'])
    # Max length of the input sequence in T5 is 512 tokens (BART could be used for longer sequences - 1024 max length limit) 
    model_input = base_tokenizer(input, max_length = 512, truncation=True, padding='max_length', return_tensors='pt')
    model_input['labels'] = base_tokenizer(label, max_length = 512, truncation=True, padding='max_length', return_tensors='pt')['input_ids']
    # model_input['decoder_input_ids'] = base_tokenizer("<pad> " + label, max_length = 512, truncation=True, padding='max_length', return_tensors='pt')['input_ids']
    return model_input
  # except:
  #   # By performing this model will also be robust to empty inputs (a type of adversarial input)
  #   input = ''
  #   label = ''
  #   model_input = base_tokenizer(input, max_length = 512, truncation=True, padding='max_length')
  #   model_input['labels'] = base_tokenizer(label, max_length = 512, truncation=True, padding='max_length')['input_ids']
  #   print("Gone here")
  #   return model_input

In [ ]:
encode_sequences(train_df.iloc[0])

In [167]:
# Loading the final dataset
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/Training_Dataset')

In [168]:
df = df[:20]

In [169]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.3)

In [170]:
# Tokenizing the dataset
train_df = train_df.apply(encode_sequences, axis=1)
test_df = test_df.apply(encode_sequences, axis = 1)

In [171]:
train_df.dropna(inplace = True)
test_df.dropna(inplace = True)

In [172]:
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

## T5 Training Setup - Custom Loss Function

In [173]:
# Initializing the Data Collator for batching of the dataset
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
        tokenizer=base_tokenizer,
        return_tensors="pt"
    )

In [12]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 5.5 MB/s eta 0:00:00


In [174]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Path where model training loss and intermediate weights will be stored
model_path = f'/content/drive/MyDrive/Visual Story Telling/Story_Gen_Model'

#Specifying the training argument 
training_args = Seq2SeqTrainingArguments(
    output_dir=model_path,
    per_device_train_batch_size=4, 
    overwrite_output_dir = True, 
    evaluation_strategy="no", 
    gradient_accumulation_steps=8, 
    num_train_epochs=1,
    weight_decay=0.01, 
    lr_scheduler_type="cosine",
    learning_rate=5e-4, 
    fp16=True 
)

In [163]:
# Overwrite the Trainer API for utilizing custom loss function 
import torch
class CustomSeq2SeqTrainer(Seq2SeqTrainer):
  def compute_loss(self, model, inputs, return_outputs=False):
      print(inputs)
      labels = inputs.pop("labels")
      input_ids = inputs.pop("input_ids")
      attention_mask = inputs.pop("attention_mask")
      print("Input_ids:", input_ids)
      print("Labels:",labels)
      print("Attention_mask:",attention_mask)
      outputs = model(input_ids = input_ids, labels = labels, attention_mask = attention_mask)
      loss_cross_entropy = outputs.loss
      logits = outputs.logits
      loss = custom_loss_function(logits) 
      return loss_cross_entropy + loss

# Check whether the tokenizer is being passed as an argument 
def custom_loss_function(logits, tokenizer=base_tokenizer):
  for each_summary in logits:
    predictions = torch.topk(each_summary,1)[1]
    predictions = [x for i in predictions for x in i]
    print("Updated:", predictions)
    generated_summary = ""
    for i in predictions:
      if i=="</s>":
        break
      generated_summary = generated_summary+" "+tokenizer.decode(i)
      print(generated_summary)
  # loss_GPT = get_chatgpt_rating(prompt4, generated_summary)
  # return loss_cross_entropy+loss_GPT
  return 0


In [175]:
# Initializing the trainer

trainer = Seq2SeqTrainer(
    model=base_model,                         # the instantiated  Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=train_df,       # training dataset
    eval_dataset = test_df,
)

# trainer = CustomSeq2SeqTrainer(
#     model=base_model,                         # the instantiated  Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     data_collator=data_collator,
#     train_dataset=train_df,       # training dataset
#     eval_dataset = test_df,
# )

In [92]:
base_model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [54]:
test_df

0    [input_ids, attention_mask, labels, decoder_in...
dtype: object

In [ ]:
print(base_model(input_ids=test_df.iloc[0].input_ids, labels=test_df.iloc[0].labels).loss)

In [94]:
print(base_model(input_ids=train_df.iloc[0].input_ids, labels=train_df.iloc[0].labels).loss)

tensor(13.3375, grad_fn=<NllLossBackward0>)


In [179]:
trainer.evaluate()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer_seq2seq.py:159 in evaluate          │
│                                                                                                  │
│   156 │   │   )                                                                                  │
│   157 │   │   self._gen_kwargs = gen_kwargs                                                      │
│   158 │   │                                                                                      │
│ ❱ 159 │   │   return super().evaluate(eval_dataset, ignore_keys=ignore_keys, metric_key_prefix   │
│   160 │                                                                                          │
│   161 │   def predict(                                                                           │
│   162 │   │   self,                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3029 in evaluate                 │
│                                                                                                  │
│   3026 │   │   start_time = time.time()                                                          │
│   3027 │   │                                                                                     │
│   3028 │   │   eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else se  │
│ ❱ 3029 │   │   output = eval_loop(                                                               │
│   3030 │   │   │   eval_dataloader,                                                              │
│   3031 │   │   │   description="Evaluation",                                                     │
│   3032 │   │   │   # No point gathering the predictions if there are no metrics, otherwise we d  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3200 in evaluation_loop          │
│                                                                                                  │
│   3197 │   │                                                                                     │
│   3198 │   │   observed_num_examples = 0                                                         │
│   3199 │   │   # Main evaluation loop                                                            │
│ ❱ 3200 │   │   for step, inputs in enumerate(dataloader):                                        │
│   3201 │   │   │   # Update the observed num examples                                            │
│   3202 │   │   │   observed_batch_size = find_batch_size(inputs)                                 │
│   3203 │   │   │   if observed_batch_size is not None:                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:633 in __next__           │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                                                      │
│    634 │   │   │   self._num_yielded += 1                  

## Model Training 

In [177]:
# Starting the training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1664 in train                    │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1909 in _inner_training_loop     │
│                                                                                                  │
│   1906 │   │   │   │   rng_to_sync = True                                                        │
│   1907 │   │   │                                                                                 │
│   1908 │   │   │   step = -1                                                                     │
│ ❱ 1909 │   │   │   for step, inputs in enumerate(epoch_iterator):                                │
│   1910 │   │   │   │   total_batched_samples += 1                                                │
│   1911 │   │   │   │   if rng_to_sync:                                                           │
│   1912 │   │   │   │   │   self._load_rng_state(resume_from_checkpoint)                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:633 in __next__           │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                                                      │
│    634 │   │   │   self._num_yielded += 1                                                        │
│    635 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    636 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:677 in _next_data         │
│                                                                                                  │
│    674 │                                                                                         │
│    675 │   def _next_data(self):                                                                 │
│    676 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  677 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    678 │   │   if self._pin_memory:                        

In [ ]:
# Saving the final model
trainer.save_model()

## Model Evaluation

In [ ]:
trainer.evaluate()

In [ ]:
# Code to get BLEU Score rating of the model on test dataset 

"""
Refer:

1) https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt
2) https://huggingface.co/docs/evaluate/choosing_a_metric

"""